In [259]:
import os
import sys
import torch
from a2c_ppo_acktr.envs import VecPyTorch, make_vec_envs
from a2c_ppo_acktr.utils import get_render_func, get_vec_normalize

sys.path.append('a2c_ppo_acktr')

In [260]:
env_names = ['HalfCheetah-v2', 'Hopper-v2', 'Ant-v2', 'Reacher-v2']
env_id = 0
env_name = env_names[env_id]
seed = 1
env = make_vec_envs(
    env_name,
    seed + 1000,
    1,
    None,
    None,
    device='cpu',
    allow_early_resets=False)

In [261]:
env

In [262]:
obs = env.reset()
print(obs)
print(type(obs))
print(obs.dtype)
obs = obs.numpy()[0]
print(obs, type(obs))

tensor([[-8.1573e-04, -9.7072e-05, -7.1744e-04,  8.7470e-04,  8.4266e-04,
          9.1401e-04,  5.4262e-04,  5.4622e-04, -1.1759e-03,  1.3422e-03,
         -4.5226e-04, -3.0781e-04,  1.3832e-03, -5.5750e-05,  2.7510e-04,
         -8.6834e-04, -1.3302e-04]])
<class 'torch.Tensor'>
torch.float32
[-8.1573415e-04 -9.7071548e-05 -7.1743538e-04  8.7470334e-04
  8.4266055e-04  9.1400777e-04  5.4262049e-04  5.4622500e-04
 -1.1758950e-03  1.3422364e-03 -4.5226331e-04 -3.0780918e-04
  1.3832008e-03 -5.5749533e-05  2.7509802e-04 -8.6833688e-04
 -1.3302304e-04] <class 'numpy.ndarray'>


In [263]:
st = env.venv.envs[0].sim.get_state()
print(st)

MjSimState(time=0.0, qpos=array([ 0.07459297, -0.08185436, -0.00970858, -0.07193608,  0.08781567,
        0.08457529,  0.09179415,  0.05434754,  0.05470964]), qvel=array([-0.11842284,  0.13546286, -0.04527718, -0.03079859,  0.13967654,
       -0.0055756 ,  0.02752297, -0.08717164, -0.01330481]), act=None, udd_state={})


In [264]:
# import gym
# env = gym.make('Hopper-v2')

print(env.observation_space.shape, env.action_space.shape)

(17,) (6,)


In [265]:
load_dir = 'trained_models/ppo'

# We need to use the same statistics for normalization as used in training
actor_critic, ob_rms = \
            torch.load(os.path.join(load_dir, env_name + ".pt"))


In [266]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

vec_norm = get_vec_normalize(env)
if vec_norm is not None:
    vec_norm.eval()
    vec_norm.ob_rms = ob_rms

recurrent_hidden_states = torch.zeros(1,
                                      actor_critic.recurrent_hidden_state_size)
masks = torch.zeros(1, 1)


In [267]:
# obs = env.reset()
# print(obs)
# print(type(obs))
# obs = torch.Tensor([obs])
# st = env.sim.get_state()
# print(obs)
# print(st)

In [ ]:
def sim_get_obs(env_id):
    if env_id == 3:
        theta = env.venv.envs[0].sim.data.qpos.flat[:2]
        return np.concatenate([
            np.cos(theta),
            np.sin(theta),
            env.venv.envs[0].sim.data.qpos.flat[2:],
            env.venv.envs[0].sim.data.qvel.flat[:2],
            env.venv.envs[0].get_body_com("fingertip") - env.venv.envs[0].get_body_com("target")
        ])
    elif env_id == 0:
        return np.concatenate([
            env.venv.envs[0].sim.data.qpos.flat[1:],
            env.venv.envs[0].sim.data.qvel.flat,
        ])
        

### Get demo trajectories (100)

In [277]:
det = True
M = 100000
trajs_demo = []
st_info = []
rew_info = []
cnt = 0
epn = 0
rews = 0

while True:
    traj = []
    traj_st = []
    traj_rew = []
    rew = 0
    
    obs = env.reset()
    st = env.venv.envs[0].sim.get_state()
    traj_st.append(st)
    obs_gym = sim_get_obs(env_id)
    while True:
        obs = obs.to(device)
        with torch.no_grad():
            value, action, _, recurrent_hidden_states = actor_critic.act(
                obs, recurrent_hidden_states, masks, deterministic=det)

        # print(type(action), action.cpu().numpy()[0])
        
        # Obser reward and next obs
        next_obs, reward, done, _ = env.step(action)
        
        # print(obs, reward, done, _)
        next_obs_gym = sim_get_obs(env_id)
        traj.append([obs_gym, action.cpu().numpy()[0], 
                     next_obs_gym])
        traj_rew.append(reward)
        
        obs = next_obs
        obs_gym = next_obs_gym
        rew += reward
        cnt += 1
        st = env.venv.envs[0].sim.get_state()
        traj_st.append(st)

        if done:
            rews += rew
            trajs_demo.append(traj)
            st_info.append(traj_st)
            rew_info.append(traj_rew)
            epn += 1
            break
            
    if cnt >= M:
        break

rews /= epn


In [278]:
print('average reward of %d demo trajs: %.2f' % (epn, rews))

average reward of 100 demo trajs: 1793.88


### Add demo triple index

In [279]:
import numpy as np

In [280]:
traj_index = 0
for traj in trajs_demo:
    triple_index = 0
    for triple in traj:
        triple.append(np.array([traj_index, triple_index]))
        triple_index += 1
    traj_index += 1

In [ ]:
# for traj in trajs_demo:
#     for triple in traj:
#         del triple[3]

In [281]:
print(len(trajs_demo[0]))
print(len(st_info[0]))
print(len(rew_info[0]))

1000
1001
1000


In [282]:
print(trajs_demo[0][0][0])
print(trajs_demo[0][0][1])
print(trajs_demo[0][1][0])
print(trajs_demo[0][1][1])
print(trajs_demo[0][2][0])
print(trajs_demo[0][2][1])
print(trajs_demo[0][3][0])
print(trajs_demo[0][3][1])
print(st_info[0][0])
print(st_info[1][0])

[-0.05514779  0.01413077  0.05431647 -0.07718844 -0.00049597 -0.09359713
  0.0420625   0.0338486  -0.04637919 -0.07595596  0.0203414   0.03111221
  0.09077952 -0.21187306  0.09768369  0.06454459 -0.02387658]
[ 4.6757808e-01  1.4283210e-03  6.5947473e-01  1.8217321e-01
  1.4451610e+00 -5.1731670e-01]
[-0.07202693 -0.01772098  0.15221099 -0.03565283  0.23121198 -0.01841621
  0.33170059 -0.1662426   0.49294502 -0.50951452 -0.75886748  2.57497647
  1.62760945  5.13046556  2.27734184  7.41693052 -5.22591411]
[ 0.54515547  0.33950102  0.29747805  0.6972709   1.1975483  -0.50355154]
[-0.11176169 -0.08448181  0.31225425  0.17700579  0.2379468   0.2849727
  0.44529777 -0.45882151  0.72204616 -1.03220343 -1.31037863  3.22701093
  4.05066736 -1.43649638  7.04318367  1.28284355 -3.77430341]
[ 0.8011819   0.23011546  0.5274805   1.0458543   1.4034928  -0.64960414]
[-0.1772751  -0.12200063  0.46834402  0.21548037  0.24988779  0.66666613
  0.49098657 -0.48806973  0.62554882 -1.45787828 -0.33301604  2

In [283]:
import pickle
file_path = '../pytorch_bco/Demo/my_demo_'+env_name+'.pkl'
with open(file_path, 'wb') as f:
    pickle.dump((trajs_demo, st_info, rew_info), f)
    f.close()

### check consistence of vec_env and gym_env

In [284]:
trajs_demo, st_info, rew_info = pickle.load(open(file_path, 'rb'))

In [285]:
import gym

env_gym = gym.make(env_name)

In [286]:
def sim_get_obs_gym(env_id):
    if env_id == 3:
        theta = env_gym.sim.data.qpos.flat[:2]
        return np.concatenate([
            np.cos(theta),
            np.sin(theta),
            env_gym.sim.data.qpos.flat[2:],
            env_gym.sim.data.qvel.flat[:2],
            env_gym.get_body_com("fingertip") - env_gym.get_body_com("target")
        ])
    elif env_id == 0:
        return np.concatenate([
            env_gym.sim.data.qpos.flat[1:],
            env_gym.sim.data.qvel.flat,
        ])
        

#### consistence of observation and action

In [287]:
cur_st = st_info[0][0]
print(cur_st)
env_gym.reset()
env_gym.sim.set_state(cur_st) #restore
obs_ = sim_get_obs_gym(env_id)
print(trajs_demo[0][0][0])
print(obs_) 
for t in range(20):
    ground_next_obs, reward, done, _ = env_gym.step(trajs_demo[0][t][1])
    print(trajs_demo[0][t][2])
    print(ground_next_obs)
    print(trajs_demo[0][t+1][0])

MjSimState(time=0.0, qpos=array([ 0.0665112 , -0.05514779,  0.01413077,  0.05431647, -0.07718844,
       -0.00049597, -0.09359713,  0.0420625 ,  0.0338486 ]), qvel=array([-0.04637919, -0.07595596,  0.0203414 ,  0.03111221,  0.09077952,
       -0.21187306,  0.09768369,  0.06454459, -0.02387658]), act=None, udd_state={})
[-0.05514779  0.01413077  0.05431647 -0.07718844 -0.00049597 -0.09359713
  0.0420625   0.0338486  -0.04637919 -0.07595596  0.0203414   0.03111221
  0.09077952 -0.21187306  0.09768369  0.06454459 -0.02387658]
[-0.05514779  0.01413077  0.05431647 -0.07718844 -0.00049597 -0.09359713
  0.0420625   0.0338486  -0.04637919 -0.07595596  0.0203414   0.03111221
  0.09077952 -0.21187306  0.09768369  0.06454459 -0.02387658]
[-0.07202693 -0.01772098  0.15221099 -0.03565283  0.23121198 -0.01841621
  0.33170059 -0.1662426   0.49294502 -0.50951452 -0.75886748  2.57497647
  1.62760945  5.13046556  2.27734184  7.41693052 -5.22591411]
[-0.07202693 -0.01772098  0.15221099 -0.03565283  0.231

#### consistence of reset function (random init distribution)

In [288]:
vec_init_observations = []
for traj in trajs_demo:
    obs = traj[0][0]
    vec_init_observations.append(obs)
    
print('number of vec_env init observations: ', len(vec_init_observations))

vec_init_O_mean = np.mean(vec_init_observations, axis=0)
vec_init_O_std = np.std(vec_init_observations, axis=0)
print('mean of vec_env init observations: ', vec_init_O_mean)
print('std of vec_env init observations: ', vec_init_O_std)

number of vec_env init observations:  100
mean of vec_env init observations:  [ 0.00792176  0.00544425 -0.00730537 -0.00221835  0.00295303 -0.00283246
  0.00562488 -0.00086885 -0.01007227 -0.00668115 -0.00397859  0.00408371
  0.00638024  0.00386069  0.01656085  0.00303774 -0.02024434]
std of vec_env init observations:  [0.05650789 0.05727206 0.05778806 0.05850251 0.05782768 0.05759043
 0.05393862 0.0538914  0.10320724 0.09602415 0.0886312  0.09992457
 0.0904926  0.08813283 0.10757881 0.09357738 0.09685105]


In [289]:
gym_init_observations = []
for traj in range(len(vec_init_observations)):
    obs = env_gym.reset()
    gym_init_observations.append(obs)
    
print('number of gym_env init observations: ', len(gym_init_observations))

gym_init_O_mean = np.mean(gym_init_observations, axis=0)
gym_init_O_std = np.std(gym_init_observations, axis=0)
print('mean of gym_env init observations: ', gym_init_O_mean)
print('std of gym_env init observations: ', gym_init_O_std)

number of gym_env init observations:  100
mean of gym_env init observations:  [-0.00884669 -0.0012371  -0.00554992 -0.00109463  0.00392025 -0.00178393
  0.0093787   0.00319286 -0.00493329  0.0203515   0.01665084  0.01611261
  0.00287764  0.00282126 -0.00626857  0.0111061  -0.00949662]
std of gym_env init observations:  [0.06256906 0.05933151 0.05870592 0.0608504  0.05404201 0.05925899
 0.05711069 0.05428466 0.09026793 0.11522247 0.0946026  0.08982903
 0.09195915 0.09988099 0.09965771 0.08958653 0.09824774]
